# CardioIA - Fase 5: Ir Além 1 - Extração de Informações com IA Generativa

## Objetivo
Utilizar técnicas de Prompt Engineering e a API do **Google Gemini** para transformar anotações médicas desestruturadas em dados estruturados (JSON), facilitando a interoperabilidade e análise de dados.

In [ ]:
!pip install google-generativeai python-dotenv pandas

In [ ]:
import os
import json
import google.generativeai as genai
from dotenv import load_dotenv
import pandas as pd

# Carrega API KEY do .env (certifique-se que o arquivo .env está na pasta raiz ou ajustado aqui)
load_dotenv('../.env') 
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    print("ERRO: Chave GEMINI_API_KEY não encontrada no .env")
else:
    genai.configure(api_key=api_key)
    print("API Gemini configurada com sucesso!")

## 1. Definição do Prompt (Engenharia de Prompt)
Criaremos um prompt do tipo "Few-Shot" (ou instrução direta robusta) para garantir que o modelo extraia exatamente os campos desejados: Nome, Idade, Sintomas, Medicamentos em uso e Sinais Vitais.

In [ ]:
def extract_clinical_data(medical_note):
    model = genai.GenerativeModel('gemini-pro')
    
    prompt = f"""
    Atue como um especialista em processamento de dados médicos.
    Sua tarefa é ler a anotação clínica abaixo e extrair as informações em formato JSON estrito.
    
    Campos obrigatórios no JSON:
    - paciente_nome (string)
    - idade (int, ou null se não informado)
    - sintomas (lista de strings)
    - medicamentos_em_uso (lista de strings)
    - sinais_vitais (objeto com pressao_arterial, frequencia_cardiaca, temperatura - use null se não houver)
    - risco_estimado (string: 'Baixo', 'Medio', 'Alto' - deduza pelo contexto)

    ANOTAÇÃO CLÍNICA:
    {medical_note}
    
    SAÍDA JSON:
    """
    
    try:
        response = model.generate_content(prompt)
        # Tratamento básico para garantir que pegamos apenas o JSON caso o modelo fale algo antes
        text_resp = response.text.replace('```json', '').replace('```', '').strip()
        return json.loads(text_resp)
    except Exception as e:
        return {"erro": str(e), "raw_response": response.text if 'response' in locals() else ""}


## 2. Teste com Caso Simulado 1

In [ ]:
caso_1 = """
Paciente Maria da Silva, 65 anos, comparece relatando forte dor no peito que irradia para o braço esquerdo, iniciada há 2 horas. 
Relata também sudorese e náusea. Nega febre. 
Faz uso contínuo de Losartana e AAS. 
Ao exame físico: PA 160/100 mmHg, FC 110 bpm, SatO2 96%.
"""

dados_1 = extract_clinical_data(caso_1)
print(json.dumps(dados_1, indent=4, ensure_ascii=False))

## 3. Teste com Caso Simulado 2 (Texto mais informal)

In [ ]:
caso_2 = """
João Souza esteve aqui hoje. O rapaz tá reclamando de uma tosse seca e as vezes falta um pouco de ar quando sobe escada. 
Ele disse que não toma remédio nenhum. A pressão tava boa, 12 por 8, e o coração batendo a 75.
"""

dados_2 = extract_clinical_data(caso_2)
print(json.dumps(dados_2, indent=4, ensure_ascii=False))

## Conclusão
O modelo Gemini foi capaz de estruturar dados complexos e heterogêneos, padronizando a saída para integração com sistemas de prontuário eletrônico ou dashboards de BI.